In [1]:
# Initial imports
import pandas as pd
import numpy as np
from pathlib import Path
from sklearn.ensemble import RandomForestClassifier
from sklearn.preprocessing import StandardScaler
from sklearn.model_selection import train_test_split
from sklearn.metrics import confusion_matrix, accuracy_score, classification_report

## Loading & Preprocessing 

In [2]:
# Import the Google Search Trend Data for the Stock Ticker into a Pandas DataFrame
amazon_search_df = pd.read_csv(Path('Amazon_Trend_Comparisons.csv'))

# Review the DataFrame for each of the Google Search Trends Data sets
amazon_search_df.head()

,Week,Amazon Profit,Amazon Loss,AMZN Buy,AMZN Sell,Amazon Stock up,Amazon Stock down,Trend
0,7/26/2020,85,69,85,31,41,10,0
1,8/2/2020,67,76,50,30,34,19,0
2,8/9/2020,59,69,42,16,37,14,0
3,8/16/2020,65,64,65,25,36,15,0
4,8/23/2020,74,63,61,11,41,24,0


## Define the Features 

In [3]:
# Define the features set by copying the Google Search Trend DataFrame
X = amazon_search_df.copy()
X.drop(columns=['Trend', 'Week'], axis=1, inplace=True)
# Review the features set of the amc_search_df 
X.head()

,Amazon Profit,Amazon Loss,AMZN Buy,AMZN Sell,Amazon Stock up,Amazon Stock down
0,85,69,85,31,41,10
1,67,76,50,30,34,19
2,59,69,42,16,37,14
3,65,64,65,25,36,15
4,74,63,61,11,41,24


## Define the Targets

In [4]:
# Create the target vector by assiging the values of the ['Column'] from the amc_search_df
y = amazon_search_df['Trend'].values.reshape(-1,1)

# Review the targets set of the carvana_search_df
y[:5]

array([[0],
       [0],
       [0],
       [0],
       [0]], dtype=int64)

## Train, Test, & Split the Features and Targets

In [5]:
# Splitting into Train and Test sets
X_train, X_test, y_train, y_test = train_test_split(X, y, random_state=70)

## Use the StandardScaler to Scale the Features Data

In [6]:
# Create the StandardScaler instance
scaler = StandardScaler()

In [7]:
# Fit the Standard Scaler with the Training Data
X_scaler = scaler.fit(X_train)

In [8]:
# Scale the Training Data
X_train_scaled = X_scaler.transform(X_train)
X_test_scaled = X_scaler.transform(X_test)

## Fitting the Random Forest Model

Once the data is scaled, create a random forest instance and train it with the training data (X_train_scaled and y_train).

In [9]:
# Create the random forest classifier instance
rf_model = RandomForestClassifier(n_estimators=500, random_state=70)

In [10]:
# Fit the model
rf_model = rf_model.fit(X_train_scaled, y_train)

C:\Users\maria\anaconda3\envs\dev\envs\dev\lib\site-packages\ipykernel_launcher.py:2: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples,), for example using ravel().
  


## Making Predictions Using the Random Forest Model

Validate the trained model, by predicting loan defaults using the testing data (X_test_scaled).

In [11]:
# Making predictions using the testing data
predictions = rf_model.predict(X_test_scaled)

## Model Evaluation

Evaluate model's results, by using sklearn to calculate the confusion matrix, the accuracy score and to generate the classification report.

In [12]:
# Calculating the confusion matrix
cm = confusion_matrix(y_test, predictions)
cm_df = pd.DataFrame(
      cm, index=["Actual 0", "Actual 1"], columns=["Predicted 0", "Predicted 1"]
)

# Calculating the accuracy score
acc_score = accuracy_score(y_test, predictions)

In [13]:
# Displaying results
print("Confusion Matrix")
display(cm_df)
print(f"Accuracy Score : {acc_score}")
print("Classification Report")
print(classification_report(y_test, predictions))

Confusion Matrix


,Predicted 0,Predicted 1
Actual 0,28,0
Actual 1,6,5


Accuracy Score : 0.8461538461538461
Classification Report
              precision    recall  f1-score   support

           0       0.82      1.00      0.90        28
           1       1.00      0.45      0.62        11

    accuracy                           0.85        39
   macro avg       0.91      0.73      0.76        39
weighted avg       0.87      0.85      0.82        39



## Feature Importance

In this section, you are asked to fetch the features' importance from the random forest model and display the top 10 most important features.

In [14]:
# Get the feature importance array
importances = rf_model.feature_importances_

In [15]:
# List the top 10 most important features
importances_sorted = sorted(zip(rf_model.feature_importances_, X.columns), reverse=True)
importances_sorted[:]

[(0.28163624728706793, 'AMZN Sell'),
 (0.21913735558588965, 'AMZN Buy'),
 (0.16175987702116498, 'Amazon Loss'),
 (0.13055923538809214, 'Amazon Stock up'),
 (0.11729711190605648, 'Amazon Profit'),
 (0.08961017281172881, 'Amazon Stock down')]

## Slice the DataFrame & Plot Against Closing Prices

In [16]:
# Change the 'week' column in the Google Search Trend DF to Datetime 
amazon_search_df['Week'] = pd.to_datetime(
    amazon_search_df['Week'],
    infer_datetime_format=True,
    utc=True
)

In [17]:
amazon_search_df.head()

,Week,Amazon Profit,Amazon Loss,AMZN Buy,AMZN Sell,Amazon Stock up,Amazon Stock down,Trend
0,2020-07-26 00:00:00+00:00,85,69,85,31,41,10,0
1,2020-08-02 00:00:00+00:00,67,76,50,30,34,19,0
2,2020-08-09 00:00:00+00:00,59,69,42,16,37,14,0
3,2020-08-16 00:00:00+00:00,65,64,65,25,36,15,0
4,2020-08-23 00:00:00+00:00,74,63,61,11,41,24,0


In [18]:
amazon_search_df['Week'] = amazon_search_df['Week'].dt.tz_localize(None)

amazon_search_df.head()

,Week,Amazon Profit,Amazon Loss,AMZN Buy,AMZN Sell,Amazon Stock up,Amazon Stock down,Trend
0,2020-07-26,85,69,85,31,41,10,0
1,2020-08-02,67,76,50,30,34,19,0
2,2020-08-09,59,69,42,16,37,14,0
3,2020-08-16,65,64,65,25,36,15,0
4,2020-08-23,74,63,61,11,41,24,0


In [19]:
# Drop every column except for the most important feature
amazon_search_df.drop(columns=['Amazon Profit', 'AMZN Buy', 'Amazon Loss', 'Amazon Stock up', 'Amazon Stock down', 'Trend'], inplace=True)

# Set the index to Week in the Google Search Trend DF
amazon_search_df.set_index('Week', inplace=True)
amazon_search_df.head()

,AMZN Sell
Week,
2020-07-26,31
2020-08-02,30
2020-08-09,16
2020-08-16,25
2020-08-23,11


In [24]:
#Get 3 Year's Worth of Stock Price Data via API
import os
import pandas as pd
import alpaca_trade_api as tradeapi
from dotenv import load_dotenv
load_dotenv()
%matplotlib inline
from alpaca_trade_api.rest import REST, TimeFrame, TimeFrameUnit

alpaca_api_key = os.getenv("ALPACA_API_KEY")
alpaca_secret_key = os.getenv("ALPACA_SECRET_KEY")
alpaca = tradeapi.REST(
alpaca_api_key,
alpaca_secret_key,
api_version="v2"
)

ticker = ['AMZN']
timeframe ='1Day'
start_date = pd.Timestamp("2018-05-01", tz="America/New_York").isoformat()
end_date = pd.Timestamp("2023-05-01", tz="America/New_York").isoformat()

df_ticker = alpaca.get_bars(
ticker,
timeframe,
start=start_date,
end=end_date
).df

AMZN = df_ticker[df_ticker['symbol']== 'AMZN'].drop('symbol', axis=1)
amazon_closing_prices = pd.DataFrame()
amazon_closing_prices["AMZN"] = AMZN["close"]




In [26]:
#Import hvplot libaries
import hvplot.pandas

#Plot the Google Search Trend DF and the Closing Prices for Comparison
amazon_search_df.hvplot() + amazon_closing_prices.hvplot()

:Layout
   .Curve.I  :Curve   [Week]   (AMZN Sell)
   .Curve.II :Curve   [timestamp]   (AMZN)

## Create A Short-Position Algorithm

In [27]:
# Calculate the daily returns using the closing prices and the pct_change function
amazon_closing_prices["actual_returns"] = amazon_closing_prices["AMZN"].pct_change()

# Drop all NaN values from the DataFrame
amazon_closing_prices = amazon_closing_prices.dropna()

# Review the DataFrame
display(amazon_closing_prices.head())
display(amazon_closing_prices.tail())

,AMZN,actual_returns
timestamp,,
2018-05-02 04:00:00+00:00,1569.680,-0.007951
2018-05-03 04:00:00+00:00,1572.075,0.001526
2018-05-04 04:00:00+00:00,1580.950,0.005645
2018-05-07 04:00:00+00:00,1600.140,0.012138
2018-05-08 04:00:00+00:00,1592.390,-0.004843


,AMZN,actual_returns
timestamp,,
2023-04-25 04:00:00+00:00,102.57,-0.034272
2023-04-26 04:00:00+00:00,104.98,0.023496
2023-04-27 04:00:00+00:00,109.82,0.046104
2023-04-28 04:00:00+00:00,105.45,-0.039792
2023-05-01 04:00:00+00:00,102.05,-0.032243


In [29]:
# Create the short_window and long_window variables
short_window = 135
long_window = 235

# Using Close Columns and Rolling & Mean Functions and Create the SMA75 and SMA145 columns for moving averages
amazon_closing_prices['SMA Short'] = amazon_closing_prices['AMZN'].rolling(window=short_window ).mean()
amazon_closing_prices['SMA Long'] = amazon_closing_prices['AMZN'].rolling(window=long_window).mean()

#Review the DataFrame
amazon_closing_prices.tail()

,AMZN,actual_returns,SMA Short,SMA Long
timestamp,,,,
2023-04-25 04:00:00+00:00,102.57,-0.034272,97.408667,217.337957
2023-04-26 04:00:00+00:00,104.98,0.023496,97.355111,208.668723
2023-04-27 04:00:00+00:00,109.82,0.046104,97.332296,200.005149
2023-04-28 04:00:00+00:00,105.45,-0.039792,97.279852,191.297915
2023-05-01 04:00:00+00:00,102.05,-0.032243,97.243926,182.578383


In [30]:
#Create the features set
#Assign a copy of the SMA25 and SMA100 columns to a new DataFrame called X
X = amazon_closing_prices[['SMA Short', 'SMA Long']].shift().dropna().copy()

#Display sample data
display(X.head())
display(X.tail())

,SMA Short,SMA Long
timestamp,,
2019-04-09 04:00:00+00:00,1682.454444,1722.964489
2019-04-10 04:00:00+00:00,1681.724667,1724.097085
2019-04-11 04:00:00+00:00,1680.782296,1725.268383
2019-04-12 04:00:00+00:00,1679.813556,1726.388043
2019-04-15 04:00:00+00:00,1678.554889,1727.421745


,SMA Short,SMA Long
timestamp,,
2023-04-25 04:00:00+00:00,97.490889,226.720085
2023-04-26 04:00:00+00:00,97.408667,217.337957
2023-04-27 04:00:00+00:00,97.355111,208.668723
2023-04-28 04:00:00+00:00,97.332296,200.005149
2023-05-01 04:00:00+00:00,97.279852,191.297915


In [31]:
#Create target set
#Create a new column in the signals_cvna called signal setting its value to zero
amazon_closing_prices["Signal"] = 0.0

In [32]:
#Create the signal to buy
amazon_closing_prices.loc[(amazon_closing_prices["actual_returns"] >= 0), "Signal"] = 1

In [33]:
#Create the signal to sell
amazon_closing_prices.loc[(amazon_closing_prices["actual_returns"] < 0), "Signal"] = -1

In [34]:
#Copy the new signal column to a new SEries called y
y = amazon_closing_prices["Signal"].copy()
y.tail()

timestamp
2023-04-25 04:00:00+00:00   -1.0
2023-04-26 04:00:00+00:00    1.0
2023-04-27 04:00:00+00:00    1.0
2023-04-28 04:00:00+00:00   -1.0
2023-05-01 04:00:00+00:00   -1.0
Name: Signal, dtype: float64

## Split the Data Into Training & Testing Datasets

### Creating the Training Datasets

In [35]:
# Imports
from sklearn.preprocessing import StandardScaler
from pandas.tseries.offsets import DateOffset

In [36]:
# Select the start of the training period
training_begin = X.index.min()

# Display the training begin date
print(training_begin)

2019-04-09 04:00:00+00:00


In [37]:
X.head()

,SMA Short,SMA Long
timestamp,,
2019-04-09 04:00:00+00:00,1682.454444,1722.964489
2019-04-10 04:00:00+00:00,1681.724667,1724.097085
2019-04-11 04:00:00+00:00,1680.782296,1725.268383
2019-04-12 04:00:00+00:00,1679.813556,1726.388043
2019-04-15 04:00:00+00:00,1678.554889,1727.421745


In [38]:
# Select the ending period for the training data with an offset of "" months
training_end = X.index.min() + DateOffset(months=15)

# Display the training end date
print(training_end)

2020-07-09 04:00:00+00:00


In [39]:
# Generate the X_train and y_train DataFrames
X_train1 = X.loc[training_begin:training_end]
y_train1 = y.loc[training_begin:training_end]

# Display sample data
X_train1.head()

,SMA Short,SMA Long
timestamp,,
2019-04-09 04:00:00+00:00,1682.454444,1722.964489
2019-04-10 04:00:00+00:00,1681.724667,1724.097085
2019-04-11 04:00:00+00:00,1680.782296,1725.268383
2019-04-12 04:00:00+00:00,1679.813556,1726.388043
2019-04-15 04:00:00+00:00,1678.554889,1727.421745


### Creating the Training Datasets

In [40]:
# Generate the X_test and y_test DataFrames
X_test1 = X.loc[training_end:]
y_test1 = y.loc[training_end:]

# Display sample data
X_test1.head()

,SMA Short,SMA Long
timestamp,,
2020-07-09 04:00:00+00:00,2195.046815,2016.965319
2020-07-10 04:00:00+00:00,2205.368444,2022.749957
2020-07-13 04:00:00+00:00,2215.229407,2028.855787
2020-07-14 04:00:00+00:00,2224.371630,2034.456511
2020-07-15 04:00:00+00:00,2233.535407,2039.948426


### Standardizing the Data

In [41]:
# Create a StandardScaler instance
scaler1 = StandardScaler()
 
# Apply the scaler model to fit the X-train data
X_scaler1 = scaler.fit(X_train1)
 
# Transform the X_train and X_test DataFrames using the X_scaler
X_train_scaled1 = X_scaler.transform(X_train1)
X_test_scaled1 = X_scaler.transform(X_test1)

## Introduce the Machine Learning Model: Logistic Regression

In [42]:
# Import LogisticRegression from sklearn
from sklearn.linear_model import LogisticRegression

In [43]:
# Create an instance of the LogisticRegression model
logistic_regression_model = LogisticRegression()

In [44]:
# Fit the LogisticRegression model
logistic_regression_model.fit(X_train_scaled1, y_train1)

LogisticRegression()

In [45]:
# Use the trained LogisticRegression model to predict the trading signals for the training data
lr_training_signal_predictions = logistic_regression_model.predict(X_train_scaled1)

# Display the predictions
lr_training_signal_predictions

array([-1., -1., -1., -1., -1., -1., -1., -1., -1., -1., -1., -1., -1.,
       -1., -1., -1., -1., -1., -1., -1., -1., -1., -1., -1., -1., -1.,
       -1., -1., -1., -1., -1., -1., -1., -1., -1., -1., -1., -1., -1.,
       -1., -1., -1., -1., -1.,  1.,  1.,  1.,  1.,  1.,  1.,  1.,  1.,
        1.,  1.,  1.,  1.,  1.,  1.,  1.,  1.,  1.,  1.,  1.,  1.,  1.,
        1.,  1.,  1.,  1.,  1.,  1.,  1.,  1.,  1.,  1.,  1.,  1.,  1.,
        1.,  1.,  1.,  1.,  1.,  1.,  1.,  1.,  1.,  1.,  1.,  1.,  1.,
        1.,  1.,  1.,  1.,  1.,  1., -1., -1., -1., -1., -1., -1., -1.,
       -1., -1., -1., -1., -1., -1., -1., -1., -1., -1., -1., -1., -1.,
       -1., -1., -1., -1., -1., -1., -1., -1., -1., -1., -1., -1., -1.,
       -1., -1., -1., -1., -1., -1., -1., -1., -1., -1., -1., -1., -1.,
        1.,  1.,  1.,  1.,  1.,  1.,  1.,  1.,  1.,  1.,  1.,  1.,  1.,
        1.,  1.,  1.,  1.,  1.,  1.,  1.,  1.,  1.,  1.,  1.,  1.,  1.,
        1.,  1.,  1.,  1.,  1.,  1.,  1.,  1.,  1.,  1.,  1.,  1

In [46]:
# Generate a classification report using the training data and the logistic regression model's predications
lr_training_report = classification_report(y_train1, lr_training_signal_predictions)

# Review the classification report
print(lr_training_report)

              precision    recall  f1-score   support

        -1.0       0.51      0.32      0.39       143
         1.0       0.57      0.75      0.65       173

    accuracy                           0.55       316
   macro avg       0.54      0.53      0.52       316
weighted avg       0.54      0.55      0.53       316



## Backtest the Logistic Regression Model 

In [47]:
# Use the trained model to predict the trading signals for the testing data.
lr_testing_signal_predictions = logistic_regression_model.predict(X_test_scaled1)

In [48]:
# Generate a classification report using the testing data and the logistic regression model's predictions
lr_testing_report = classification_report(y_test1, lr_testing_signal_predictions)

# Review the testing classification report
print(lr_testing_report)

              precision    recall  f1-score   support

        -1.0       0.52      0.18      0.26       358
         1.0       0.50      0.83      0.62       350

    accuracy                           0.50       708
   macro avg       0.51      0.50      0.44       708
weighted avg       0.51      0.50      0.44       708



In [49]:
# Create a new empty predictions DataFrame using code provided below.
predictions_df = pd.DataFrame(index=X_test1.index)
predictions_df["predicted_signal"] = lr_testing_signal_predictions
predictions_df["actual_returns"] = amazon_closing_prices["actual_returns"]
predictions_df["trading_algorithm_returns"] = predictions_df["actual_returns"] * predictions_df["predicted_signal"]
predictions_df.head()
predictions_df.tail()

,predicted_signal,actual_returns,trading_algorithm_returns
timestamp,,,
2023-04-25 04:00:00+00:00,-1.0,-0.034272,0.034272
2023-04-26 04:00:00+00:00,-1.0,0.023496,-0.023496
2023-04-27 04:00:00+00:00,-1.0,0.046104,-0.046104
2023-04-28 04:00:00+00:00,-1.0,-0.039792,0.039792
2023-05-01 04:00:00+00:00,-1.0,-0.032243,0.032243


In [50]:
#Calcualte the cumulative returns for the actual returns and trading algorithm returns
cumulative_df = pd.DataFrame(index=X_test1.index)
cumulative_df['Actual Cumulative Returns'] = (1 + predictions_df[['actual_returns']]).cumprod()
cumulative_df['Algo Cumulative Returns'] = (1 + predictions_df[['trading_algorithm_returns']]).cumprod()

#Review the cumulative returns dataframe
cumulative_df.tail()

,Actual Cumulative Returns,Algo Cumulative Returns
timestamp,,
2023-04-25 04:00:00+00:00,0.033290,0.024829
2023-04-26 04:00:00+00:00,0.034072,0.024245
2023-04-27 04:00:00+00:00,0.035643,0.023128
2023-04-28 04:00:00+00:00,0.034225,0.024048
2023-05-01 04:00:00+00:00,0.033121,0.024823


In [51]:
#Plot the Google Search Trends in Comparison to the Cumulative Returns
amazon_search_df.hvplot() + cumulative_df.hvplot()

:Layout
   .Curve.I     :Curve   [Week]   (AMZN Sell)
   .NdOverlay.I :NdOverlay   [Variable]
      :Curve   [timestamp]   (value)

# Back Test the Actual Trading Strategy


In [53]:
#Create the dataframe for the Actual Cumualtive Returns
actual_backtest = pd.concat([amazon_closing_prices, cumulative_df], axis='columns', join='inner')

#Drop the Algo Cumulative Returns Column
actual_backtest.drop(columns='Algo Cumulative Returns', inplace=True)

# Calculate the points in time at which a position should be taken, 1 or -1
actual_backtest['Entry/Exit'] = actual_backtest['Signal'].diff()
pd.set_option('display.max_rows', None)

#Review the dataframe
actual_backtest.tail()

,AMZN,actual_returns,SMA Short,SMA Long,Signal,Actual Cumulative Returns,Entry/Exit
timestamp,,,,,,,
2023-04-25 04:00:00+00:00,102.57,-0.034272,97.408667,217.337957,-1.0,0.033290,0.0
2023-04-26 04:00:00+00:00,104.98,0.023496,97.355111,208.668723,1.0,0.034072,2.0
2023-04-27 04:00:00+00:00,109.82,0.046104,97.332296,200.005149,1.0,0.035643,0.0
2023-04-28 04:00:00+00:00,105.45,-0.039792,97.279852,191.297915,-1.0,0.034225,-2.0
2023-05-01 04:00:00+00:00,102.05,-0.032243,97.243926,182.578383,-1.0,0.033121,0.0


In [54]:
# Set the intial capital 
initial_capital = float(100000)

# Set the share size
share_size = 500

#Take the share position where the dual moving average crossover is 1
actual_backtest['Position'] = share_size * actual_backtest['Signal']

# Find the points in time where a 500 share position is bought or sold
actual_backtest["Entry/Exit Position"] = actual_backtest["Position"].diff()

# Multiply share price by entry/exit positions and get the cumulatively sum
actual_backtest["Portfolio Holdings"] = (
    actual_backtest["AMZN"] * actual_backtest["Entry/Exit Position"].cumsum()
)

# Subtract the initial capital by the portfolio holdings to get the amount of liquid cash in the portfolio
actual_backtest["Portfolio Cash"] = (
    initial_capital - (actual_backtest["AMZN"] * actual_backtest["Entry/Exit Position"]).cumsum()
)

# Get the total portfolio value by adding the cash amount by the portfolio holdings (or investments)
actual_backtest["Portfolio Total"] = (
   actual_backtest["Portfolio Cash"] + actual_backtest["Portfolio Holdings"]
)

# Calculate the portfolio daily returns
actual_backtest["Portfolio Daily Returns"] = actual_backtest["Portfolio Total"].pct_change()

# Calculate the cumulative returns
actual_backtest["Portfolio Cumulative Returns"] = (
    1 + actual_backtest["Portfolio Daily Returns"]
).cumprod() - 1

# Print the DataFrame
actual_backtest.tail()

,AMZN,actual_returns,SMA Short,SMA Long,Signal,Actual Cumulative Returns,Entry/Exit,Position,Entry/Exit Position,Portfolio Holdings,Portfolio Cash,Portfolio Total,Portfolio Daily Returns,Portfolio Cumulative Returns
timestamp,,,,,,,,,,,,,,
2023-04-25 04:00:00+00:00,102.57,-0.034272,97.408667,217.337957,-1.0,0.033290,0.0,-500.0,0.0,-102570.0,3659370.0,3556800.0,0.001024,34.5680
2023-04-26 04:00:00+00:00,104.98,0.023496,97.355111,208.668723,1.0,0.034072,2.0,500.0,1000.0,0.0,3554390.0,3554390.0,-0.000678,34.5439
2023-04-27 04:00:00+00:00,109.82,0.046104,97.332296,200.005149,1.0,0.035643,0.0,500.0,0.0,0.0,3554390.0,3554390.0,0.000000,34.5439
2023-04-28 04:00:00+00:00,105.45,-0.039792,97.279852,191.297915,-1.0,0.034225,-2.0,-500.0,-1000.0,-105450.0,3659840.0,3554390.0,0.000000,34.5439
2023-05-01 04:00:00+00:00,102.05,-0.032243,97.243926,182.578383,-1.0,0.033121,0.0,-500.0,0.0,-102050.0,3659840.0,3557790.0,0.000957,34.5779


## Plot Entry/Exit Points of Actual Backtest Results

In [56]:
# Visualize entry postions relative to closing price
entry  = actual_backtest[actual_backtest['Entry/Exit'] == 2.0]['AMZN'].hvplot.scatter(
    color='purple',
    marker='^',
    legend=False,
    ylabel='Price in $',
    width=1000,
    height=400)

# Visualize exit positions relative to close price
exit = actual_backtest[actual_backtest['Entry/Exit'] == -2.0]['AMZN'].hvplot.scatter(
    color='orange',
    marker='v',
    legend=False,
    ylabel='Price in $',
    width=1000,
    height=400)

# Visualize the close price for the investment
security_close = actual_backtest[['AMZN']].hvplot(
    line_color='lightgray',
    ylabel='Price in $',
    width=1000,
    height=400
)

# Visualize the moving averages
moving_avgs = actual_backtest[['SMA Short', 'SMA Long']].hvplot(
    ylabel='Price in $',
    width=1000,
    height=400
)

# Overlay the plots
portfolio_entry_exit_plot = security_close * moving_avgs * entry * exit
portfolio_entry_exit_plot.opts(
    title="VAQ Short-Position Actual - Total Portfolio Value",
    yformatter='%.0f'
) + amazon_search_df.hvplot(width=1000,
    height=400)

:Layout
   .Overlay.I :Overlay
      .Curve.I         :Curve   [timestamp]   (AMZN)
      .NdOverlay.I     :NdOverlay   [Variable]
         :Curve   [timestamp]   (value)
      .Scatter.AMZN.I  :Scatter   [timestamp]   (AMZN)
      .Scatter.AMZN.II :Scatter   [timestamp]   (AMZN)
   .Curve.I   :Curve   [Week]   (AMZN Sell)

# Back Test the Algorithmic Trading Strategy

In [57]:
#Create the dataframe for the Actual Cumualtive Returns
algo_backtest = pd.concat([amazon_closing_prices, cumulative_df], axis='columns', join='inner')

#Drop the Algo Cumulative Returns Column
algo_backtest.drop(columns= 'Actual Cumulative Returns', inplace=True)

# Calculate the points in time at which a position should be taken, 1 or -1
algo_backtest['Entry/Exit'] = algo_backtest['Signal'].diff()
pd.set_option('display.max_rows', None)

#Review the dataframe
algo_backtest.tail()

,AMZN,actual_returns,SMA Short,SMA Long,Signal,Algo Cumulative Returns,Entry/Exit
timestamp,,,,,,,
2023-04-25 04:00:00+00:00,102.57,-0.034272,97.408667,217.337957,-1.0,0.024829,0.0
2023-04-26 04:00:00+00:00,104.98,0.023496,97.355111,208.668723,1.0,0.024245,2.0
2023-04-27 04:00:00+00:00,109.82,0.046104,97.332296,200.005149,1.0,0.023128,0.0
2023-04-28 04:00:00+00:00,105.45,-0.039792,97.279852,191.297915,-1.0,0.024048,-2.0
2023-05-01 04:00:00+00:00,102.05,-0.032243,97.243926,182.578383,-1.0,0.024823,0.0


In [59]:
# Set the intial capital 
initial_capital = float(100000)

# Set the share size
share_size = 500

#Take the share position where the dual moving average crossover is 1
algo_backtest['Position'] = share_size * algo_backtest['Signal']

# Find the points in time where a 500 share position is bought or sold
algo_backtest["Entry/Exit Position"] = algo_backtest["Position"].diff()

# Multiply share price by entry/exit positions and get the cumulatively sum
algo_backtest["Portfolio Holdings"] = (
    algo_backtest["AMZN"] * algo_backtest["Entry/Exit Position"].cumsum()
)

# Subtract the initial capital by the portfolio holdings to get the amount of liquid cash in the portfolio
algo_backtest["Portfolio Cash"] = (
    initial_capital - (algo_backtest["AMZN"] * algo_backtest["Entry/Exit Position"]).cumsum()
)

# Get the total portfolio value by adding the cash amount by the portfolio holdings (or investments)
algo_backtest["Portfolio Total"] = (
   algo_backtest["Portfolio Cash"] + algo_backtest["Portfolio Holdings"]
)

# Calculate the portfolio daily returns
algo_backtest["Portfolio Daily Returns"] = algo_backtest["Portfolio Total"].pct_change()

# Calculate the cumulative returns
algo_backtest["Portfolio Cumulative Returns"] = (
    1 + algo_backtest["Portfolio Daily Returns"]
).cumprod() - 1

# Print the DataFrame
algo_backtest.tail()

,AMZN,actual_returns,SMA Short,SMA Long,Signal,Algo Cumulative Returns,Entry/Exit,Position,Entry/Exit Position,Portfolio Holdings,Portfolio Cash,Portfolio Total,Portfolio Daily Returns,Portfolio Cumulative Returns
timestamp,,,,,,,,,,,,,,
2023-04-25 04:00:00+00:00,102.57,-0.034272,97.408667,217.337957,-1.0,0.024829,0.0,-500.0,0.0,-102570.0,3659370.0,3556800.0,0.001024,34.5680
2023-04-26 04:00:00+00:00,104.98,0.023496,97.355111,208.668723,1.0,0.024245,2.0,500.0,1000.0,0.0,3554390.0,3554390.0,-0.000678,34.5439
2023-04-27 04:00:00+00:00,109.82,0.046104,97.332296,200.005149,1.0,0.023128,0.0,500.0,0.0,0.0,3554390.0,3554390.0,0.000000,34.5439
2023-04-28 04:00:00+00:00,105.45,-0.039792,97.279852,191.297915,-1.0,0.024048,-2.0,-500.0,-1000.0,-105450.0,3659840.0,3554390.0,0.000000,34.5439
2023-05-01 04:00:00+00:00,102.05,-0.032243,97.243926,182.578383,-1.0,0.024823,0.0,-500.0,0.0,-102050.0,3659840.0,3557790.0,0.000957,34.5779


## Plot Entry/Exit Points of Actual Backtest Results

In [60]:
# Visualize entry postions relative to closing price
entry1 = algo_backtest[algo_backtest['Entry/Exit'] == 2.0]['AMZN'].hvplot.scatter(
    color='purple',
    marker='^',
    legend=False,
    ylabel='Price in $',
    width=1000,
    height=400)

# Visualize exit positions relative to close price
exit1 = algo_backtest[algo_backtest['Entry/Exit'] == -2.0]['AMZN'].hvplot.scatter(
    color='orange',
    marker='v',
    legend=False,
    ylabel='Price in $',
    width=1000,
    height=400)

# Visualize the close price for the investment
security_close1 = algo_backtest[['AMZN']].hvplot(
    line_color='lightgray',
    ylabel='Price in $',
    width=1000,
    height=400
)

# Visualize the moving averages
moving_avgs1 = algo_backtest[['SMA Short', 'SMA Long']].hvplot(
    ylabel='Price in $',
    width=1000,
    height=400
)

# Overlay the plots
portfolio_entry_exit_plot1 = security_close1 * moving_avgs1 * entry1 * exit1
portfolio_entry_exit_plot1.opts(
    title="VAQ Short-Position Algorithmic - Total Portfolio Value",
    yformatter='%.0f'
) + amazon_search_df.hvplot(width=1000,
    height=400) 

:Layout
   .Overlay.I :Overlay
      .Curve.I         :Curve   [timestamp]   (AMZN)
      .NdOverlay.I     :NdOverlay   [Variable]
         :Curve   [timestamp]   (value)
      .Scatter.AMZN.I  :Scatter   [timestamp]   (AMZN)
      .Scatter.AMZN.II :Scatter   [timestamp]   (AMZN)
   .Curve.I   :Curve   [Week]   (AMZN Sell)

## Actual Portfolio Evaluation Metrics

In [61]:
# Create the list of the metric names
metrics = [
    'Annualized Return',
    'Cumulative Returns',
    'Annual Volatility',
    'Sharpe Ratio',
    'Sortino Ratio'
]

# Create a list that holds the column name
columns = ['Backtest']

# Initialize the DataFrame with index set to evaluation metrics and columns 
actual_evaluation_df = pd.DataFrame(index=metrics, columns=columns)

# Review the DataFrame
actual_evaluation_df

,Backtest
Annualized Return,NaN
Cumulative Returns,NaN
Annual Volatility,NaN
Sharpe Ratio,NaN
Sortino Ratio,NaN


In [62]:
# Calculate the Annualized return metric
actual_evaluation_df.loc['Annualized Return'] = (
    actual_backtest['Portfolio Daily Returns'].mean() * 252
)

In [63]:
# Calculate the Cumulative returns metric
actual_evaluation_df.loc['Cumulative Returns'] = actual_backtest['Portfolio Cumulative Returns'][-1]

In [64]:
# Calculate the Annual volatility metric
actual_evaluation_df.loc['Annual Volatility'] = (
    actual_backtest['Portfolio Daily Returns'].std() * np.sqrt(252)
)

In [65]:
# Calculate the Sharpe ratio
actual_evaluation_df.loc['Sharpe Ratio'] = (
    actual_backtest['Portfolio Daily Returns'].mean() * 252) / (
    actual_backtest['Portfolio Daily Returns'].std() * np.sqrt(252)
)

In [66]:
# Calculate the Sortino ratio
# Start by calculating the downside return values

# Create a DataFrame that contains the Portfolio Daily Returns column
sortino_ratio_df = actual_backtest[['Portfolio Daily Returns']].copy()

# Create a column to hold downside return values
sortino_ratio_df.loc[:,'Downside Returns'] = 0

# Find Portfolio Daily Returns values less than 0, 
# square those values, and add them to the Downside Returns column
sortino_ratio_df.loc[sortino_ratio_df['Portfolio Daily Returns'] < 0, 
                     'Downside Returns'] = sortino_ratio_df['Portfolio Daily Returns']**2

# Calculate the annualized return value
annualized_return = sortino_ratio_df['Portfolio Daily Returns'].mean() * 252

# Calculate the annualized downside standard deviation value
downside_standard_deviation = np.sqrt(sortino_ratio_df['Downside Returns'].mean()) * np.sqrt(252)

# Divide the annualized return value by the downside standard deviation value
sortino_ratio = annualized_return/downside_standard_deviation

# Add the Sortino ratio to the evaluation DataFrame
actual_evaluation_df.loc['Sortino Ratio'] = sortino_ratio

In [67]:
# Review the portfolio evaluation DataFrame
actual_evaluation_df

,Backtest
Annualized Return,-11.240002
Cumulative Returns,34.5779
Annual Volatility,39.130301
Sharpe Ratio,-0.287245
Sortino Ratio,-0.30545


##  Acutal Trade Trade Evaluation

In [68]:
# Initialize the trade evaluation DataFrame
actual_evaluation_df = pd.DataFrame(
    columns=[
        'Stock', 
        'Entry Date', 
        'Exit Date', 
        'Shares', 
        'Entry Share Price', 
        'Exit Share Price', 
        'Entry Portfolio Holding', 
        'Exit Portfolio Holding', 
        'Profit/Loss']
)

In [71]:
# Initialize the iterative variables
entry_date = ""
exit_date = ""
entry_portfolio_holding = 0.0
exit_portfolio_holding = 0.0
share_size = 0
entry_share_price = 0.0
exit_share_price = 0.0

# Loop through the signal DataFrame
# If 'Entry/Exit' is 2,s et entry trade metrics
# Else if 'Entry/Exit' is -2, set exit trade metrics and calcualte profit,
for index, row in actual_backtest.iterrows():
    if row['Entry/Exit'] == 2:
        entry_date = index
        entry_portfolio_holding = abs(row['Portfolio Holdings'])
        share_size = row['Entry/Exit Position']
        entry_share_price = row['AMZN']
        
    elif row['Entry/Exit'] == -2:
        exit_date = index
        exit_portfolio_holding = abs(row['AMZN'] * row['Entry/Exit Position'])
        exit_share_price = row['AMZN']
        profit_loss =  entry_portfolio_holding - exit_portfolio_holding
        actual_evaluation_df = actual_evaluation_df.append(
            {
                'Stock': 'AMZN',
                'Entry Date': entry_date,
                'Exit Date': exit_date,
                'Shares': share_size,
                'Entry Share Price': entry_share_price,
                'Exit Share Price': exit_share_price,
                'Entry Portfolio Holding': entry_portfolio_holding,
                'Exit Portfolio Holding': exit_portfolio_holding,
                'Profit/Loss': profit_loss
            },
            ignore_index=True)
        
# Review the DataFrame
actual_evaluation_df.tail()

,Stock,Entry Date,Exit Date,Shares,Entry Share Price,Exit Share Price,Entry Portfolio Holding,Exit Portfolio Holding,Profit/Loss
167,AMZN,2023-04-06 04:00:00+00:00,2023-04-11 04:00:00+00:00,1000.0,102.06,99.92,0.0,99920.0,-99920.0
168,AMZN,2023-04-13 04:00:00+00:00,2023-04-18 04:00:00+00:00,1000.0,102.40,102.30,0.0,102300.0,-102300.0
169,AMZN,2023-04-19 04:00:00+00:00,2023-04-20 04:00:00+00:00,1000.0,104.30,103.81,0.0,103810.0,-103810.0
170,AMZN,2023-04-21 04:00:00+00:00,2023-04-24 04:00:00+00:00,1000.0,106.96,106.21,0.0,106210.0,-106210.0
171,AMZN,2023-04-26 04:00:00+00:00,2023-04-28 04:00:00+00:00,1000.0,104.98,105.45,0.0,105450.0,-105450.0


## Algorithmic Portfolio Evaluation Metrics

In [72]:
# Create the list of the metric names
metrics1 = [
    'Annualized Return',
    'Cumulative Returns',
    'Annual Volatility',
    'Sharpe Ratio',
    'Sortino Ratio'
]

# Create a list that holds the column name
columns1 = ['Backtest']

# Initialize the DataFrame with index set to evaluation metrics and columns 
algo_evaluation_df = pd.DataFrame(index=metrics1, columns=columns1)

# Review the DataFrame
algo_evaluation_df

,Backtest
Annualized Return,NaN
Cumulative Returns,NaN
Annual Volatility,NaN
Sharpe Ratio,NaN
Sortino Ratio,NaN


In [73]:
algo_evaluation_df.loc['Annualized Return'] = (
    algo_backtest['Portfolio Daily Returns'].mean() * 252
)

In [74]:
# Calculate the Cumulative returns metric
algo_evaluation_df.loc['Cumulative Returns'] = algo_backtest['Portfolio Cumulative Returns'][-1]

In [75]:
# Calculate the Annual volatility metric
algo_evaluation_df.loc['Annual Volatility'] = (
    algo_backtest['Portfolio Daily Returns'].std() * np.sqrt(252)
)

In [76]:
# Calculate the Sharpe ratio
algo_evaluation_df.loc['Sharpe Ratio'] = (
    algo_backtest['Portfolio Daily Returns'].mean() * 252) / (
    algo_backtest['Portfolio Daily Returns'].std() * np.sqrt(252)
)

In [77]:
# Calculate the Sortino ratio
# Start by calculating the downside return values

# Create a DataFrame that contains the Portfolio Daily Returns column
sortino_ratio_df = algo_backtest[['Portfolio Daily Returns']].copy()

# Create a column to hold downside return values
sortino_ratio_df.loc[:,'Downside Returns'] = 0

# Find Portfolio Daily Returns values less than 0, 
# square those values, and add them to the Downside Returns column
sortino_ratio_df.loc[sortino_ratio_df['Portfolio Daily Returns'] < 0, 
                     'Downside Returns'] = sortino_ratio_df['Portfolio Daily Returns']**2

# Calculate the annualized return value
annualized_return = sortino_ratio_df['Portfolio Daily Returns'].mean() * 252

# Calculate the annualized downside standard deviation value
downside_standard_deviation = np.sqrt(sortino_ratio_df['Downside Returns'].mean()) * np.sqrt(252)

# Divide the annualized return value by the downside standard deviation value
sortino_ratio = annualized_return/downside_standard_deviation

# Add the Sortino ratio to the evaluation DataFrame
algo_evaluation_df.loc['Sortino Ratio'] = sortino_ratio


In [78]:
# Review the portfolio evaluation DataFrame
algo_evaluation_df

,Backtest
Annualized Return,-11.240002
Cumulative Returns,34.5779
Annual Volatility,39.130301
Sharpe Ratio,-0.287245
Sortino Ratio,-0.30545


##  Algo Trade Trade Evaluation

In [79]:
# Initialize the trade evaluation DataFrame
algo_evaluation_df = pd.DataFrame(
    columns=[
        'Stock', 
        'Entry Date', 
        'Exit Date', 
        'Shares', 
        'Entry Share Price', 
        'Exit Share Price', 
        'Entry Portfolio Holding', 
        'Exit Portfolio Holding', 
        'Profit/Loss']
)

In [80]:
# Initialize the iterative variables
entry_date = ""
exit_date = ""
entry_portfolio_holding = 0.0
exit_portfolio_holding = 0.0
share_size = 0
entry_share_price = 0.0
exit_share_price = 0.0

# Loop through the signal DataFrame
# If 'Entry/Exit' is 1,s et entry trade metrics
# Else if 'Entry/Exit' is -2, set exit trade metrics and calcualte profit,
for index, row in algo_backtest.iterrows():
    if row['Entry/Exit'] == 2:
        entry_date = index
        entry_portfolio_holding = abs(row['Portfolio Holdings'])
        share_size = row['Entry/Exit Position']
        entry_share_price = row['AMZN']
        
    elif row['Entry/Exit'] == -2:
        exit_date = index
        exit_portfolio_holding = abs(row['AMZN'] * row['Entry/Exit Position'])
        exit_share_price = row['AMZN']
        profit_loss =  entry_portfolio_holding - exit_portfolio_holding
        algo_evaluation_df = algo_evaluation_df.append(
            {
                'Stock': 'ANZN',
                'Entry Date': entry_date,
                'Exit Date': exit_date,
                'Shares': share_size,
                'Entry Share Price': entry_share_price,
                'Exit Share Price': exit_share_price,
                'Entry Portfolio Holding': entry_portfolio_holding,
                'Exit Portfolio Holding': exit_portfolio_holding,
                'Profit/Loss': profit_loss
            },
            ignore_index=True)
        
# Review the DataFrame
algo_evaluation_df.tail()

,Stock,Entry Date,Exit Date,Shares,Entry Share Price,Exit Share Price,Entry Portfolio Holding,Exit Portfolio Holding,Profit/Loss
166,ANZN,2023-04-06 04:00:00+00:00,2023-04-11 04:00:00+00:00,1000.0,102.06,99.92,0.0,99920.0,-99920.0
167,ANZN,2023-04-13 04:00:00+00:00,2023-04-18 04:00:00+00:00,1000.0,102.40,102.30,0.0,102300.0,-102300.0
168,ANZN,2023-04-19 04:00:00+00:00,2023-04-20 04:00:00+00:00,1000.0,104.30,103.81,0.0,103810.0,-103810.0
169,ANZN,2023-04-21 04:00:00+00:00,2023-04-24 04:00:00+00:00,1000.0,106.96,106.21,0.0,106210.0,-106210.0
170,ANZN,2023-04-26 04:00:00+00:00,2023-04-28 04:00:00+00:00,1000.0,104.98,105.45,0.0,105450.0,-105450.0
